In [11]:
!pip install dash
!pip install dash-bootstrap-components
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 4.1 MB/s eta 0:00:00


In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

In [15]:
# Deployed at: https://fifa-world-cup-dashboard.onrender.com (no password required)

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server

# Load and preprocess the data
df = pd.read_csv('data.csv')

# Define country to ISO code mappings
country_to_code = {
    'Uruguay': 'URU',
    'Italy': 'ITA',
    'Germany': 'DEU',
    'Brazil': 'BRA',
    'England': 'ENG',
    'Argentina': 'ARG',
    'France': 'FRA',
    'Spain': 'ESP'
}

# Fill missing 'Winner_Country' using the mapping
df['Winner_Country'] = df['Winner_Country'].fillna(df['Winner'].map(country_to_code))

# Prepare data for Choropleth map
win_counts = df['Winner_Country'].value_counts().reset_index()
win_counts.columns = ['Country_Code', 'Wins']

# Create the Choropleth figure
choropleth_fig = px.choropleth(
    win_counts,
    locations='Country_Code',
    color='Wins',
    hover_name='Country_Code',
    color_continuous_scale='Viridis',
    title='FIFA World Cup Wins by Country'
)

# Define app layout
app.layout = html.Div([
    html.H1("FIFA World Cup Winners Dashboard", style={'textAlign': 'center'}),
    dcc.Graph(figure=choropleth_fig),

    html.H3("All Winning Countries"),
    html.Ul([html.Li(country) for country in df['Winner'].unique()], id='winning-countries-list'),

    html.H3("Select a Country to View Number of Wins"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in df['Winner'].unique()],
        placeholder="Select a country..."
    ),
    html.Div(id='win-count-output', style={'marginTop': 10}),

    html.H3("Select a Year to View Finalists"),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in df['Year'].unique()],
        placeholder="Select a year..."
    ),
    html.Div(id='year-output', style={'marginTop': 10})
])

# Callback to display number of wins for selected country
@app.callback(
    Output('win-count-output', 'children'),
    Input('country-dropdown', 'value')
)
def update_win_count(selected_country):
    if not selected_country:
        return "Select a country to see the number of wins."
    count = df[df['Winner'] == selected_country].shape[0]
    return f"{selected_country} has won {count} times."

# Callback to display finalists for selected year
@app.callback(
    Output('year-output', 'children'),
    Input('year-dropdown', 'value')
)
def update_year_info(selected_year):
    if not selected_year:
        return "Select a year to see the finalists."
    data = df[df['Year'] == selected_year].iloc[0]
    return html.Div([
        html.P(f"Winner: {data['Winner']} ({data['Winner_Country']})"),
        html.P(f"Runner-Up: {data['Runner_Up']} ({data['Runner_Up_Country']})")
    ])

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>